# SHIFT_ML_2026_COMPETITION

## Описание задачи

Ссылка на архив с данными

Данные представляют собой более миллиона записей с более чем 100 признаками и одной целевой переменной: «итоговый_статус_займа». Значения: 0 – выплачен, 1 – не выплачен. Задача – бинарная классификация.

В вашем распоряжении будет два датасета: один тренировочный (с целевой переменной), и один тестовый – без целевой переменной.

Также в архиве находится ноутбук baseline.ipynb с примером подготовки файла с ответом.

Постановка задачи
Вам предстоит помочь банку принять решение о выдаче займа клиентам. Помните, что одного лишь принятого решения мало – нужно уметь интерпретировать полученные результаты.

## Импорт библиотек

In [51]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import Ridge
from catboost import CatBoostClassifier

## Загрузка данных

In [52]:
test_path = './shift_ml_2026_test.csv'
train_path = './shift_ml_2026_train.csv'
test_df = pd.read_csv(test_path)
train_df = pd.read_csv(train_path)
print('train')
print(train_df.shape)
display(train_df.head())
print('test')
print(test_df.shape)
display(test_df.head())
print('y_train')
print(train_df['итоговый_статус_займа'].value_counts())

C:\Temp\ipykernel_13320\2014380920.py:4: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_path)


train
(1210779, 109)


,id,сумма_займа,срок_займа,процентная_ставка,аннуитет,рейтинг,допрейтинг,профессия_заемщика,стаж,владение_жильем,...,процент_счетов_прев_75_лимита,кол-во_публ_банкротств,кол-во_залогов,кредитный_лимит,кредитный_баланс_без_ипотеки,лимит_по_картам,лимит_по_аннуитетным_счетам,кредитный_баланс_по_возоб_счетам,особая_ситуация,тип_предоставления_кредита
0,68355089,1235000.0,3 года,11.99,41014.0,В,В1,инженер,10+ лет,ИПОТЕКА,...,7.7,0.0,0.0,15700850.0,1973750.0,3965000.0,1233350.0,NaN,Нет,Наличные
1,68341763,1000000.0,5 лет,10.78,21633.0,Б,Б4,водитель грузовика,10+ лет,ИПОТЕКА,...,50.0,0.0,0.0,10920900.0,934800.0,310000.0,743850.0,NaN,Нет,Наличные
2,68426831,597500.0,3 года,13.44,20259.0,В,В3,ветеринарный техник,4 года,АРЕНДА,...,100.0,0.0,0.0,845000.0,639900.0,470000.0,200000.0,NaN,Нет,Наличные
3,68476668,1000000.0,3 года,9.17,31879.0,Б,Б2,вице-президент операций по набору персонала,10+ лет,ИПОТЕКА,...,100.0,0.0,0.0,19442600.0,5838100.0,1575000.0,2322600.0,NaN,Нет,Наличные
4,67275481,1000000.0,3 года,8.49,31563.0,Б,Б1,дорожному водителю,10+ лет,ИПОТЕКА,...,0.0,0.0,0.0,9669500.0,1396850.0,725000.0,1807200.0,NaN,Нет,Наличные


test
(134531, 108)


,id,сумма_займа,срок_займа,процентная_ставка,аннуитет,рейтинг,допрейтинг,профессия_заемщика,стаж,владение_жильем,...,процент_счетов_прев_75_лимита,кол-во_публ_банкротств,кол-во_залогов,кредитный_лимит,кредитный_баланс_без_ипотеки,лимит_по_картам,лимит_по_аннуитетным_счетам,кредитный_баланс_по_возоб_счетам,особая_ситуация,тип_предоставления_кредита
0,85540387,450000.0,3 года,9.49,14413.0,Б,Б2,обслуживание клиентов,10+ лет,ИПОТЕКА,...,75.0,0.0,0.0,4282850.0,1180600.0,725000.0,1022000.0,NaN,Нет,Наличные
1,28112500,400000.0,3 года,6.03,12174.5,А,А1,помощник по правовым вопросам,5 лет,АРЕНДА,...,57.1,0.0,0.0,3340900.0,2381050.0,1260000.0,1548550.0,NaN,Нет,Наличные
2,65731570,1250000.0,3 года,12.05,41548.0,В,В1,специалист по анализу кредитоспособности,5 лет,ИПОТЕКА,...,16.7,0.0,0.0,11350200.0,1980650.0,1690000.0,1804450.0,NaN,Нет,Наличные
3,65874747,977500.0,5 лет,20.99,26439.5,Д,Д5,специальный специалист,3 года,ИПОТЕКА,...,66.7,1.0,0.0,9976550.0,1678600.0,470000.0,2139950.0,NaN,Нет,Наличные
4,57893355,520000.0,3 года,18.25,18865.0,Д,Д1,руководитель районного проекта,3 года,АРЕНДА,...,66.7,0.0,0.0,1953950.0,1272750.0,1125000.0,603950.0,NaN,Нет,Наличные


y_train
итоговый_статус_займа
0    969085
1    241694
Name: count, dtype: int64


## Предобработка данных

Разбиение на `train` и `val`

In [53]:
target = 'итоговый_статус_займа'
y = train_df[target]
X = train_df.drop(columns=[target])

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


### Доля пропусков в данных

Рассчитываем долю пропущенных значений как для тренировочной, так и для тестовой выборок. 

Сортируем по убыванию доли пропущенных значений

In [54]:
missing_train = X_train.isna().mean()
missing_test  = test_df.isna().mean()

missing_table = pd.DataFrame({
    "train": missing_train,
    "test": missing_test
})

missing_table["diff"] = (missing_table.train - missing_table.test).abs()
with pd.option_context("display.max_rows", None, 
                       "display.max_columns", None,
                       "display.float_format", '{:.4f}'.format):
    display(missing_table.sort_values("train", ascending=False).head(30))

,train,test,diff
дата_следующей_выплаты,1.0000,1.0000,0.0000
кредитный_баланс_по_возоб_счетам,0.9862,0.9865,0.0003
совокупный_статус_подтверждения_доходов_заемщиков,0.9811,0.9811,0.0000
совокупный_пдн_заемщиков,0.9809,0.9809,0.0000
совокупный_доход_заемщиков,0.9809,0.9809,0.0000
кол-во_месяцев_с_последнего_займа,0.8301,0.8298,0.0003
кол-во_мес_с_последней_задолженности_по_карте,0.7627,0.7639,0.0013
кол-во_месяцев_с_последнего_нарушения,0.7369,0.7392,0.0023
кол-во_мес_с_последней_задолженности_по_возобновляемому_счету,0.6653,0.6671,0.0018
соотношение_сумм_текущего_баланса_к_лимиту_по_аннуитетным_счетам,0.6545,0.6534,0.0011


Как мы можем видеть, доля пропущенных значений изменяется от 0 до 1, что закономерно. На данный момент сложно сказать, что делать с пропусками, анализ этого момента будет ниже. Есть столбцы, которые нужно явно удалить:
* `дата_следующей_выплаты` - нет значений
* `пени_за_дефолт` - нет значений в `test`

Удаление полей, которые не несут в себе полезной информации для модели, т. е.:
* Все значения пропущены в `train` или в `test`
* Сильно различается доля пропусков в `train` и `test`

In [55]:
drop_cols = []

for col in X_train.columns:
    if missing_train[col] == 1:
        drop_cols.append(col)
    elif col in missing_test and abs(missing_train[col]-missing_test[col]) > 0.3:
        drop_cols.append(col)
    elif X_train[col].nunique() <= 1:
        drop_cols.append(col)

X_train = X_train.drop(columns=drop_cols)
X_val   = X_val.drop(columns=drop_cols)
test_df = test_df.drop(columns=drop_cols)

print("Удалено:", len(drop_cols))
for drop_col in drop_cols:
    print(drop_col)


Удалено: 7
платежный_график
коэфф_невыплаченного_сумм_остатка
непогашенная_сумма_из_тела_займов
пени_за_дефолт
дата_следующей_выплаты
код_политики
особая_ситуация


### Категориальные поля

Выводим для каждого категориального столбца его
* Название
* Число уникальных значений
* Число пропусков
* Топ 5 самых частых значений

In [56]:
cat_cols = X_train.select_dtypes(exclude='number').columns
for cat_col in cat_cols:
    data = X_train[cat_col]
    print(cat_col)
    print('Уникальных значений', len(data.unique()))
    print(data.unique()[:5])
    print('Пропущенных значений', data.isna().sum())
    print(data.value_counts(ascending=False).head(5))
    print()
print(target, len(y_train.unique()))
print(y_train.unique())
print()

срок_займа
Уникальных значений 2
['5 лет' '3 года']
Пропущенных значений 0
срок_займа
3 года    734600
5 лет     234023
Name: count, dtype: int64

рейтинг
Уникальных значений 7
['В' 'Б' 'Д' 'А' 'Г']
Пропущенных значений 0
рейтинг
Б    282750
В    275200
А    169139
Г    144497
Д     67411
Name: count, dtype: int64

допрейтинг
Уникальных значений 35
['В1' 'Б1' 'Д2' 'А5' 'А4']
Пропущенных значений 0
допрейтинг
В1    61464
Б4    59989
Б5    59524
Б3    58833
В2    57202
Name: count, dtype: int64

профессия_заемщика
Уникальных значений 229882
['ответственный за бухгалтерский учет' 'федеральный агент'
 'медицинское обслуживание'
 'ит специалист по анализу оперативной деятельности' 'менеджер ит']
Пропущенных значений 61739
профессия_заемщика
менеджер         18678
преподаватель    17892
владелец         11232
медсестра        11178
водитель          9162
Name: count, dtype: int64

стаж
Уникальных значений 12
['10+ лет' '3 года' '2 года' '< 1 года' '7 лет']
Пропущенных значений 56482
стаж
10+

Заполним пропуски в категориальных столбцах, где это возможно и необходимо. Пропуски есть в полях:
* `профессия заемщика` - обработке поля посвящен отдельный раздел
* `стаж` -> варианта 2: заполняем либо `< 1 года`, либо особенным значением `unknown`. Лучше заполним `unknown`, т.к. иначе сильно исказим распределение значений.
* `совокупный_статус_подтверждения_доходов_заемщиков` - тут по смыслу `Не подтвержден`, т.к. у нас нет информации

In [57]:
for df in [X_train, X_val, test_df]:
    df['стаж'] = df['стаж'].fillna('unknown')
    df['совокупный_статус_подтверждения_доходов_заемщиков'] = df['совокупный_статус_подтверждения_доходов_заемщиков'].fillna('Не подтвержден')

Проверка

In [58]:
print(X_train['стаж'].value_counts())
print(X_train['совокупный_статус_подтверждения_доходов_заемщиков'].value_counts())

стаж
10+ лет     318552
2 года       87486
< 1 года     78087
3 года       77451
1 год        63755
5 лет        60407
4 года       57917
unknown      56482
6 лет        45115
8 лет        43768
7 лет        42930
9 лет        36673
Name: count, dtype: int64
совокупный_статус_подтверждения_доходов_заемщиков
Не подтвержден          960888
Подтвержден источник      4507
Подтвержден               3228
Name: count, dtype: int64


Большинство столбцов действительно категориальные, которые имеет смысл использовать для обучения модели, но есть исключения:
* `профессия заемщика` - почти 230000 уникальных значений, есть более и менее популярные, но в любом случае поле сложно использовать, т.к. каждая отдельная профессия встречается не более чем в 20000 записей из 1200000 (менее 2% для самой популярной профессии). Возможно закодировать только часть профессий, поскольку поле может оказаться важным для модели.
* `дата_первого_займа` - это дата, нужно привести к типу даты

In [59]:
display(X_train['профессия_заемщика'].value_counts().sample(20))

профессия_заемщика
министерство юстиции                                47
социальная работа                                   30
прямой работник ftbs                                 1
испытание на управление полетами                     1
судебные услуги                                      1
стааб-менеджмент                                     1
аутентикс                                            1
машина установлена                                   7
профессор университета                               5
специалист по управлению медицинской информацией     1
hot food cook and deli worker                        1
дополнительный директор                              1
c & s whesale groceers                               1
свинец имплеентатино                                 1
- трудящийся, работающий на дюймах;                  1
геометрический инженер                               1
потребитель недвижимости                             2
зарегистрированный полисомнограф-технолог     

Можем видеть смешение разного написания профессий, ролей, отраслей, работодателей и мусора.
* синонимы:

    + менеджер, менеджер sr, менеджер магазина, менеджер операций

    + инженер, инженер сети, инженер проекта, инженер-механик

* разные языки:

    + manager, vp, sr, qa, it

    + русские переводы + английские оригиналы

* уровни должностей вместо профессий:

    + старший, вице-президент, директор

    + это роль, не профессия

* отрасли вместо профессий:

    + финансы

    + маркетинг

    + образование

* работодатели вместо профессий:

    + ibm

    + boeing

    + walmart

    + банк америки

* мусор:

    + html-артефакты

    + странные токены

    + обрезанные слова

    + опечатки

#### Работа с полем `профессия заемщика`

Нормализация профессий

In [60]:
# -----------------------------
# Расширенная нормализация профессий
# -----------------------------

eng_map = {
    "sr": "старший",
    "jr": "младший",
    "vp": "вице президент",
    "mgr": "менеджер",
    "qa": "качество",
    "it": "ит",
    "rn": "медсестра",
    "lpn": "медсестра",
    "cna": "медсестра",
    "driver": "водитель",
    "engineer": "инженер",
    "developer": "разработчик",
    "teacher": "учитель",
    "sales": "продажи",
}

def normalize_profession(s):
    if pd.isna(s):
        return "unknown"
    
    s = str(s).lower()
    
    # html / мусор
    s = re.sub(r'<.*?>', ' ', s)
    s = re.sub(r'[^a-zа-я0-9\s\-]', ' ', s)
    
    # замены англ сокращений
    for k, v in eng_map.items():
        s = re.sub(rf'\b{k}\b', v, s)
    
    s = re.sub(r'\s+', ' ', s).strip()
    
    if len(s) == 0:
        return "unknown"
        
    return s


for df in [X_train, X_val, test_df]:
    df['профессия_заемщика'] = df['профессия_заемщика'].apply(normalize_profession)


Выделение работодателей отдельно

In [61]:
# -----------------------------
# Детектор компаний
# -----------------------------

company_keywords = r"ibm|boeing|walmart|verizon|bank|банк|corp|inc|ltd|ooo|llc"

def is_company(s):
    return int(bool(re.search(company_keywords, s)))

for df in [X_train, X_val, test_df]:
    df["prof_is_company"] = df["профессия_заемщика"].apply(is_company)


Группы профессий

In [62]:
profession_groups = {
    "медицина": r"мед|врач|терапевт|стоматолог|фармацевт|медсестра|клиник",
    "образование": r"учител|преподавател|профессор|лектор",
    "it": r"программист|разработчик|данных|аналитик|qa|тестировщик|девопс",
    "инженерия": r"инженер|механик|техник|электрик",
    "финансы": r"бухгалтер|банкир|финанс|кредит|аудитор",
    "менеджмент": r"менеджер|директор|руководитель|вице президент",
    "продажи": r"продаж|торгов|sales",
    "маркетинг": r"маркетинг|реклама|pr",
    "юристы": r"адвокат|юрист|legal",
    "госслужба": r"гос|муницип|департамент",
    "силовые": r"полици|армия|военн|офицер|шериф",
    "транспорт": r"водител|перевоз|дальнобой",
    "строительство": r"строит|прораб|плотник",
    "производство": r"завод|фабрик|станок|оператор",
    "логистика": r"склад|кладовщик|погрузчик",
    "сервис": r"официант|бармен|кассир|повар|уборщик",
    "административные": r"секретар|администратор|клерк",
    "наука": r"scientist|исследователь",
    "бизнес": r"владелец|предприниматель|self employed"
}


Функция маппинга

In [63]:
def map_prof_group(s):
    if re.search(company_keywords, s):
        return "company_employee"
    
    for group, pattern in profession_groups.items():
        if re.search(pattern, s):
            return group
            
    return "other"


for df in [X_train, X_val, test_df]:
    df['prof_group'] = df['профессия_заемщика'].apply(map_prof_group)


Анализ редких профессий

In [64]:
freq = X_train['профессия_заемщика'].value_counts()

RARE_THRESHOLD = 50
rare_prof = set(freq[freq < RARE_THRESHOLD].index)

def mark_rare(s, g):
    if s in rare_prof and g == "other":
        return "rare_other"
    return s

for df in [X_train, X_val, test_df]:
    df['prof_clean'] = df.apply(
        lambda r: mark_rare(r['профессия_заемщика'], r['prof_group']),
        axis=1
    )


Частотный encoding профессии

In [65]:
prof_freq_map = X_train['prof_clean'].value_counts(normalize=True)

for df in [X_train, X_val, test_df]:
    df['prof_freq_enc'] = df['prof_clean'].map(prof_freq_map).fillna(0)


In [66]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np


def kfold_target_encoding(
    train_col,
    y,
    test_col,
    n_splits=5,
    smoothing=20,
    noise=0.0,
    random_state=42
):
    """
    KFold Target Encoding с защитой от leakage
    
    train_col — категориальный признак train (pd.Series)
    y         — таргет train (pd.Series)
    test_col  — тот же признак в val/test
    
    smoothing — сила сглаживания (20–100 обычно хорошо)
    noise     — добавка шума для регуляризации (0–0.01)
    """

    train_col = train_col.astype(str)
    test_col  = test_col.astype(str)

    global_mean = y.mean()

    kf = StratifiedKFold(
        n_splits=n_splits,
        shuffle=True,
        random_state=random_state
    )

    train_encoded = pd.Series(index=train_col.index, dtype=float)

    # ---------- OOF encoding ----------
    for tr_idx, val_idx in kf.split(train_col, y):

        tr_cat = train_col.iloc[tr_idx]
        tr_y   = y.iloc[tr_idx]

        stats = tr_y.groupby(tr_cat).agg(['mean', 'count'])

        smooth = (
            stats['mean'] * stats['count'] +
            global_mean * smoothing
        ) / (stats['count'] + smoothing)

        val_map = train_col.iloc[val_idx].map(smooth)

        train_encoded.iloc[val_idx] = val_map.fillna(global_mean)

    # ---------- full train stats for test ----------
    stats_full = y.groupby(train_col).agg(['mean', 'count'])

    smooth_full = (
        stats_full['mean'] * stats_full['count'] +
        global_mean * smoothing
    ) / (stats_full['count'] + smoothing)

    test_encoded = test_col.map(smooth_full).fillna(global_mean)

    # ---------- optional noise ----------
    if noise > 0:
        train_encoded *= (1 + noise * np.random.randn(len(train_encoded)))
        test_encoded  *= (1 + noise * np.random.randn(len(test_encoded)))

    return train_encoded, test_encoded


Частотный encoding по группе

In [67]:
group_freq_map = X_train['prof_group'].value_counts(normalize=True)

for df in [X_train, X_val, test_df]:
    df['prof_group_freq'] = df['prof_group'].map(group_freq_map)


In [68]:
# -----------------------------
# Target encoding по группе профессии
# -----------------------------

X_train['prof_group_te'], X_val['prof_group_te'] = kfold_target_encoding(
    X_train['prof_group'],
    y_train,
    X_val['prof_group'],
    smoothing=50,
    noise=0.01
)

_, test_df['prof_group_te'] = kfold_target_encoding(
    X_train['prof_group'],
    y_train,
    test_df['prof_group']
)


Target encoding по профессии

In [69]:
X_train['prof_te'], X_val['prof_te'] = kfold_target_encoding(
    X_train['prof_clean'],
    y_train,
    X_val['prof_clean'],
    smoothing=50,
    noise=0.01
)

_, test_df['prof_te'] = kfold_target_encoding(
    X_train['prof_clean'],
    y_train,
    test_df['prof_clean'],
    smoothing=50
)


Удаление промежуточных полей

In [70]:
drop_prof_cols = [
   # 'профессия_заемщика',
    'prof_clean'
]

X_train = X_train.drop(columns=drop_prof_cols)
X_val   = X_val.drop(columns=drop_prof_cols)
test_df = test_df.drop(columns=drop_prof_cols)


В итоге получили:
* `prof_group`           ← отрасль
* `prof_group_te`        ← риск отрасли
* `prof_group_freq`      ← размер отрасли
* `prof_te`              ← риск профессии
* `prof_freq_enc`        ← частота профессии
* `prof_is_company`      ← флаг работодателя

Обновление списка категориальных полей

In [71]:
cat_cols = X_train.select_dtypes(exclude='number').columns

Проверка новых полей

In [72]:
print('Средняя вероятность дефолта =', y_train.mean())
display(X_train[['prof_group', 'prof_group_te', 'prof_group_freq',\
    'prof_te', 'prof_freq_enc', 'prof_is_company']].head(15))

Средняя вероятность дефолта = 0.19961842739641739


,prof_group,prof_group_te,prof_group_freq,prof_te,prof_freq_enc,prof_is_company
938963,финансы,0.172627,0.025341,0.213718,0.000147,0
108797,other,0.202078,0.537837,0.144043,0.000063,0
287505,медицина,0.198643,0.045688,0.261577,0.000142,0
403826,other,0.201620,0.537837,0.152682,0.000067,0
484031,менеджмент,0.190538,0.154114,0.125523,0.001251,0
85910,медицина,0.201851,0.045688,0.200547,0.000231,0
613914,other,0.201418,0.537837,0.205399,0.000104,0
325319,транспорт,0.263525,0.027912,0.291424,0.000817,0
1077974,менеджмент,0.190992,0.154114,0.158683,0.000041,0
692939,other,0.200102,0.537837,0.188617,0.278487,0


In [73]:
X_train['prof_group'].value_counts(normalize=True)


prof_group
other               0.537837
менеджмент          0.154114
инженерия           0.050426
медицина            0.045688
образование         0.030509
транспорт           0.027912
финансы             0.025341
продажи             0.022925
it                  0.016442
бизнес              0.014539
производство        0.013670
company_employee    0.013281
административные    0.011498
силовые             0.010109
сервис              0.007085
госслужба           0.006444
юристы              0.003957
маркетинг           0.003386
строительство       0.002179
логистика           0.002152
наука               0.000506
Name: proportion, dtype: float64

In [74]:
X_train.loc[X_train.prof_group == "other", "профессия_заемщика"] \
    .value_counts() \
    .head(50)

профессия_заемщика
unknown                                           61744
рн                                                 6869
помощник по административным вопросам              5855
генеральный управляющий                            3974
президент                                          3676
вице-президент                                     3325
управляющий счетом                                 2531
форман                                             2183
сервер                                             2164
технический сотрудник                              1775
консультант                                        1711
параюридический                                    1678
техническое обслуживание                           1652
служба обслуживания клиентов                       1621
контролер                                          1613
исполнительный счет                                1561
суперинтендант                                     1502
сотрудник исправительных учре

Изменение типа поля `дата_первого_займа`

In [75]:
for df in [X_train, X_val, test_df]:
    df['дата_первого_займа'] = pd.to_datetime(
        df['дата_первого_займа'],
        format='%m-%Y',
        errors='coerce'
    )

    df['first_loan_year']  = df['дата_первого_займа'].dt.year
    df['first_loan_month'] = df['дата_первого_займа'].dt.month

    df.drop(columns=['дата_первого_займа'], inplace=True)

Добавление флагов для полей с пропусками

In [76]:
high_missing = missing_train[missing_train > 0.2].index

def add_missing_flags(df, cols):
    df = df.copy()
    for c in cols:
        if c in df:
            df[c+"_miss"] = df[c].isna().astype(int)
    return df

X_train = add_missing_flags(X_train, high_missing)
X_val   = add_missing_flags(X_val, high_missing)
test_df = add_missing_flags(test_df, high_missing)


### Непрерывные поля

Заполнение числовых полей по смыслу:
* '%месяц%' -> 999
* '%кол-во%' -> 0
* остальные -> медиана

In [77]:
month_cols = [c for c in X_train.columns if "месяц" in c]

for c in month_cols:
    X_train[c] = X_train[c].fillna(999)
    X_val[c]   = X_val[c].fillna(999)
    test_df[c] = test_df[c].fillna(999)

In [78]:
count_cols = [c for c in X_train.columns if "кол-во" in c]

for c in count_cols:
    X_train[c] = X_train[c].fillna(0)
    X_val[c]   = X_val[c].fillna(0)
    test_df[c] = test_df[c].fillna(0)


In [79]:
num_cols = X_train.select_dtypes(include='number').columns
cat_cols = X_train.select_dtypes(exclude='number').columns
med = X_train[num_cols].median()

X_train[num_cols] = X_train[num_cols].fillna(med)
X_val[num_cols]   = X_val[num_cols].fillna(med)
test_df[num_cols] = test_df[num_cols].fillna(med)


Функция для расчета Information Value

In [80]:
def calc_iv(df, feature, target, bins=10):
    d = df[[feature, target]].copy()
    
    try:
        if d[feature].nunique() > bins:
            d["bin"] = pd.qcut(d[feature], bins, duplicates='drop')
        else:
            d["bin"] = d[feature]
    except:
        return 0
    
    g = d.groupby("bin")[target].agg(["count","sum"])
    g.columns = ["total","bad"]
    g["good"] = g.total - g.bad
    
    g["bad_rate"]  = g.bad / g.bad.sum()
    g["good_rate"] = g.good / g.good.sum()
    
    g["woe"] = np.log((g.good_rate+1e-6)/(g.bad_rate+1e-6))
    g["iv"]  = (g.good_rate-g.bad_rate)*g.woe
    
    return g.iv.sum()


Вычисляем IV для числовых признаков

In [81]:
tmp = X_train.copy()
tmp[target] = y_train

iv_scores = {}

for col in X_train.columns:
    if col in cat_cols:
        continue
    iv_scores[col] = calc_iv(tmp, col, target)

iv_series = pd.Series(iv_scores).sort_values(ascending=False);
display(iv_series.head(30))


C:\Temp\ipykernel_13320\3531124842.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin")[target].agg(["count","sum"])
C:\Temp\ipykernel_13320\3531124842.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin")[target].agg(["count","sum"])
C:\Temp\ipykernel_13320\3531124842.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin")[target].agg(["count","sum"])
C:\Temp\ip

процентная_ставка                                    0.444614
сумма_выплат_по_просрочкам                           0.211605
нижний_порог_рейтинга_заемщика                       0.122058
верхний_порог_рейтинга_заемщика                      0.122058
пдн                                                  0.072216
кол-во_открытых_счетов_за_2_года                     0.066176
суммарная_доступная_сумма_займа_по_картам            0.053564
средний_баланс_текущих_счетов                        0.046855
кол-во_счетов_за_посл_год                            0.046131
prof_te                                              0.044227
кредитный_лимит                                      0.042718
лимит_по_картам                                      0.036020
общая_сумма_на_счетах                                0.035654
сумма_займа                                          0.034848
кол-во_активных_возобновляемых_счетов                0.034826
кол-во_возобновляемых_счетов_с_балансом_более_0      0.033318
кол-во_м

Отбор по IV

In [82]:
iv_selected = iv_series[iv_series > 0.01].index.tolist()
print("IV selected:", len(iv_selected))


IV selected: 57


Отбор по корреляции

In [83]:
corr = X_train[iv_selected].corr().abs()

upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

to_drop_corr = [
    column for column in upper.columns
    if any(upper[column] > 0.9)
]

final_features = [c for c in iv_selected if c not in to_drop_corr]

print("После корреляции:", len(final_features))


После корреляции: 38


In [84]:
num_cols = X_train.select_dtypes(include='number').columns
cat_cols = X_train.select_dtypes(exclude='number').columns
print('Всего полей', len(X_train.columns))
print('Числовых полей', len(num_cols))
print('Категориальных полей', len(cat_cols))
for col in X_train.columns:
    print(col, pd.api.types.is_numeric_dtype(X_train[col]))

Всего полей 131
Числовых полей 115
Категориальных полей 16
id True
сумма_займа True
срок_займа False
процентная_ставка True
аннуитет True
рейтинг False
допрейтинг False
профессия_заемщика False
стаж False
владение_жильем False
годовой_доход True
подтвержден_ли_доход False
рейтинг_OCR True
коэфф_акционных_зачислений_в_остатке True
цель_займа False
кол_во_субсидий_за_год True
индекс_проживания True
регион False
пдн True
пос_стоп_фактор False
юридический_статус False
кол-во_просрочек_за_два_года True
коэфф_исходного_платежа True
нижний_порог_рейтинга_заемщика True
верхний_порог_рейтинга_заемщика True
кол-во_заявок_за_полгода True
кол-во_месяцев_с_последней_просрочки True
кол-во_месяцев_с_последнего_займа True
кол-во_открытых_счетов True
кол-во_записей_в_общедоступных_документах True
оборотный_баланс True
коэфф_загрузки_возобновляемого_счета True
кол-во_счетов True
первоначальный_статус_займа False
сумма_выплат_по_просрочкам True
грейд_на_детерминаторе True
кол-во_взысканий_за_последний_го

Вспомогательные списки

In [85]:
# -----------------------------
# Признаки профессии
# -----------------------------
prof_num = [
    'prof_group_te',
    'prof_group_freq',
    'prof_te',
    'prof_freq_enc',
    'prof_is_company'
]

prof_cat = ['prof_group']

# Проверка наличия
for c in prof_num + prof_cat:
    assert c in X_train.columns, f"{c} not in X_train"


Проверка типов полей для профессии

In [86]:
X_train[prof_num + prof_cat].info()

<class 'pandas.core.frame.DataFrame'>
Index: 968623 entries, 938963 to 636513
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   prof_group_te    968623 non-null  float64
 1   prof_group_freq  968623 non-null  float64
 2   prof_te          968623 non-null  float64
 3   prof_freq_enc    968623 non-null  float64
 4   prof_is_company  968623 non-null  int64  
 5   prof_group       968623 non-null  object 
dtypes: float64(4), int64(1), object(1)
memory usage: 51.7+ MB


Типы верные

In [87]:
X_train[X_train.select_dtypes(exclude='number').columns].info()

<class 'pandas.core.frame.DataFrame'>
Index: 968623 entries, 938963 to 636513
Data columns (total 16 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   срок_займа                                         968623 non-null  object
 1   рейтинг                                            968623 non-null  object
 2   допрейтинг                                         968623 non-null  object
 3   профессия_заемщика                                 968623 non-null  object
 4   стаж                                               968623 non-null  object
 5   владение_жильем                                    968623 non-null  object
 6   подтвержден_ли_доход                               968623 non-null  object
 7   цель_займа                                         968623 non-null  object
 8   регион                                             968623 non-null  object
 9   пос_

In [88]:
X_train[final_features].info()

<class 'pandas.core.frame.DataFrame'>
Index: 968623 entries, 938963 to 636513
Data columns (total 38 columns):
 #   Column                                                            Non-Null Count   Dtype  
---  ------                                                            --------------   -----  
 0   процентная_ставка                                                 968623 non-null  float64
 1   сумма_выплат_по_просрочкам                                        968623 non-null  float64
 2   нижний_порог_рейтинга_заемщика                                    968623 non-null  float64
 3   пдн                                                               968623 non-null  float64
 4   кол-во_открытых_счетов_за_2_года                                  968623 non-null  float64
 5   суммарная_доступная_сумма_займа_по_картам                         968623 non-null  float64
 6   средний_баланс_текущих_счетов                                     968623 non-null  float64
 7   кол-во_счетов_за_пос

Формируем наборы признаков

In [89]:
# -----------------------------
# Feature sets
# -----------------------------
cat_cols = X_train.select_dtypes(exclude='number').columns

# A — без профессии
features_A_num = [feature for feature in final_features if feature not in prof_num]
features_A_cat = [c for c in cat_cols if c not in prof_cat]
print('Пересечение признаков A =',set(features_A_num) & set(features_A_cat))
features_A = []
for fAn in features_A_num:
    features_A.append(fAn)
for fAc in features_A_cat:
    features_A.append(fAc)

# B — с профессией (полный)
features_B_num = final_features + [col for col in prof_num if col not in final_features]
features_B_cat = list(cat_cols.copy())
print('Пересечение признаков B =',set(features_B_num) & set(features_B_cat))
features_B = []
for fBn in features_B_num:
    features_B.append(fBn)
for fBc in features_B_cat:
    features_B.append(fBc)

# C — минимальный набор профессии
features_C_num = final_features + [col for col in [
    'prof_group_te',
    'prof_te',
    'prof_freq_enc'
] if col not in final_features]
features_C_cat = ['prof_group']
print('Пересечение признаков C =',set(features_C_num) & set(features_C_cat))

print(len(features_A_cat), len(features_A_num), len(features_A_cat)+len(features_A_num))
print(len(features_B_cat), len(features_B_num), len(features_B_cat)+len(features_B_num))
print(len(features_C_cat), len(features_C_num), len(features_C_cat)+len(features_C_num))


Пересечение признаков A = set()
Пересечение признаков B = set()
Пересечение признаков C = set()
15 35 50
16 40 56
1 38 39


Проверка на наличие пропусков

In [90]:
print('Пропуски в наборе признаков B')
display(X_train[features_B]\
    .isna().sum().sort_values(ascending=False).head())

Пропуски в наборе признаков B


процентная_ставка                           0
сумма_выплат_по_просрочкам                  0
кол-во_месяцев_с_посл_аннуитетного_счета    0
кол-во_открытых_счетов_за_полгода           0
кол-во_активных_карт                        0
dtype: int64

In [91]:
X_train[features_B].info()

<class 'pandas.core.frame.DataFrame'>
Index: 968623 entries, 938963 to 636513
Data columns (total 56 columns):
 #   Column                                                            Non-Null Count   Dtype  
---  ------                                                            --------------   -----  
 0   процентная_ставка                                                 968623 non-null  float64
 1   сумма_выплат_по_просрочкам                                        968623 non-null  float64
 2   нижний_порог_рейтинга_заемщика                                    968623 non-null  float64
 3   пдн                                                               968623 non-null  float64
 4   кол-во_открытых_счетов_за_2_года                                  968623 non-null  float64
 5   суммарная_доступная_сумма_займа_по_картам                         968623 non-null  float64
 6   средний_баланс_текущих_счетов                                     968623 non-null  float64
 7   кол-во_счетов_за_пос

Отлично, пропусков нет

### Проверка наличия явных дупликатов

In [92]:
X = pd.concat([X_train, X_val], axis=0)
y = pd.concat([y_train, y_val], axis=0)
display(X.duplicated().sum())

0

Явные дупликаты отсутствуют

### Проверка типов полей

In [93]:
final_num_features = final_features + [col for col in prof_num if col not in final_features]
final_cat_features = list(cat_cols.copy())
features_to_print = final_num_features + final_cat_features
for i, feature in enumerate(features_to_print):
    print(i+1,feature, X_train[feature].dtype)
    print(X_train[feature].unique()[:5])
    print()

1 процентная_ставка float64
[14.3   8.18 18.84  7.89 23.88]

2 сумма_выплат_по_просрочкам float64
[   0.   818.5 2136.  1174.5 3004.5]

3 нижний_порог_рейтинга_заемщика float64
[715. 670. 725. 710. 730.]

4 пдн float64
[14.63 18.34 31.39  4.65 28.59]

5 кол-во_открытых_счетов_за_2_года float64
[2. 8. 1. 7. 6.]

6 суммарная_доступная_сумма_займа_по_картам float64
[ 342850.  147650.   45800.   68000. 2353250.]

7 средний_баланс_текущих_счетов float64
[130800. 795750. 483750. 154700. 267100.]

8 кол-во_счетов_за_посл_год float64
[2. 1. 5. 6. 4.]

9 prof_te float64
[0.21371772 0.14404288 0.26157688 0.15268165 0.12552338]

10 кредитный_лимит float64
[ 2012350. 10724250.  8056150.  1809200.  6274800.]

11 лимит_по_картам float64
[ 930000. 1070000.  550000.   90000. 3335000.]

12 сумма_займа float64
[ 600000.  260000. 1200000.  150000. 1275000.]

13 кол-во_активных_возобновляемых_счетов float64
[ 6.  9. 10.  2.  4.]

14 кол-во_месяцев_с_последнего_счета float64
[6. 3. 7. 4. 5.]

15 id int64
[

Функция для обучения CatBoost

In [94]:
def train_cb(
    X_tr, y_tr,
    X_va, y_va,
    num_feats,
    cat_feats,
    name=""
):
    features = num_feats + cat_feats

    model = CatBoostClassifier(
        iterations=800,
        learning_rate=0.05,
        depth=6,
        l2_leaf_reg=5,
        loss_function='Logloss',
        eval_metric='AUC',
        random_seed=42,
        verbose=100,
        early_stopping_rounds=50
    )

    model.fit(
        X_tr[features],
        y_tr,
        eval_set=(X_va[features], y_va),
        cat_features=cat_feats,
        use_best_model=True
    )

    val_pred = model.predict_proba(X_va[features])[:, 1]
    auc = roc_auc_score(y_va, val_pred)

    print(f"\n{name} AUC = {auc:.5f}")

    return model, auc


Обучение модели

In [95]:
# -----------------------------
# Train models
# -----------------------------

model_A, auc_A = train_cb(
    X_train, y_train,
    X_val, y_val,
    features_A_num,
    features_A_cat,
    name="A: no profession"
)


0:	test: 0.7258498	best: 0.7258498 (0)	total: 1.01s	remaining: 13m 25s
100:	test: 0.7526665	best: 0.7526665 (100)	total: 1m 16s	remaining: 8m 47s
200:	test: 0.7575702	best: 0.7575702 (200)	total: 2m 31s	remaining: 7m 31s
300:	test: 0.7599233	best: 0.7599233 (300)	total: 3m 48s	remaining: 6m 19s
400:	test: 0.7614659	best: 0.7614659 (400)	total: 5m 17s	remaining: 5m 15s
500:	test: 0.7625369	best: 0.7625369 (500)	total: 6m 39s	remaining: 3m 58s
600:	test: 0.7632788	best: 0.7632788 (600)	total: 8m 5s	remaining: 2m 40s
700:	test: 0.7639362	best: 0.7639362 (700)	total: 9m 34s	remaining: 1m 21s


KeyboardInterrupt: 

In [ ]:
model_B, auc_B = train_cb(
    X_train, y_train,
    X_val, y_val,
    features_B_num,
    features_B_cat,
    name="B: full profession"
)

0:	test: 0.7251395	best: 0.7251395 (0)	total: 1.37s	remaining: 18m 12s
100:	test: 0.7532989	best: 0.7532989 (100)	total: 1m 23s	remaining: 9m 40s
200:	test: 0.7580278	best: 0.7580278 (200)	total: 2m 48s	remaining: 8m 22s
300:	test: 0.7604702	best: 0.7604702 (300)	total: 4m 13s	remaining: 6m 59s
400:	test: 0.7621937	best: 0.7621937 (400)	total: 5m 35s	remaining: 5m 34s
500:	test: 0.7633491	best: 0.7633491 (500)	total: 7m	remaining: 4m 10s
600:	test: 0.7640910	best: 0.7640910 (600)	total: 8m 24s	remaining: 2m 46s
700:	test: 0.7645142	best: 0.7645142 (700)	total: 9m 51s	remaining: 1m 23s
799:	test: 0.7649121	best: 0.7649121 (799)	total: 11m 17s	remaining: 0us

bestTest = 0.7649120536
bestIteration = 799


B: full profession AUC = 0.76491


In [ ]:
model_C, auc_C = train_cb(
    X_train, y_train,
    X_val, y_val,
    features_C_num,
    features_C_cat,
    name="C: minimal profession"
)

0:	test: 0.7134666	best: 0.7134666 (0)	total: 302ms	remaining: 4m 1s
100:	test: 0.7471226	best: 0.7471226 (100)	total: 26.8s	remaining: 3m 5s
200:	test: 0.7502980	best: 0.7502980 (200)	total: 54.5s	remaining: 2m 42s
300:	test: 0.7519042	best: 0.7519042 (300)	total: 1m 21s	remaining: 2m 15s
400:	test: 0.7529640	best: 0.7529640 (400)	total: 1m 49s	remaining: 1m 49s
500:	test: 0.7537082	best: 0.7537082 (500)	total: 2m 17s	remaining: 1m 21s
600:	test: 0.7542092	best: 0.7542106 (598)	total: 2m 43s	remaining: 54.3s
700:	test: 0.7546012	best: 0.7546012 (700)	total: 3m 10s	remaining: 26.9s
799:	test: 0.7549030	best: 0.7549030 (799)	total: 3m 37s	remaining: 0us

bestTest = 0.7549030057
bestIteration = 799


C: minimal profession AUC = 0.75490
